In [ ]:
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import numpy as np
import matplotlib
matplotlib.rcParams["image.interpolation"] = None
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from glob import glob
from tifffile import imread, imsave
from csbdeep.utils import Path, normalize
from csbdeep.io import save_tiff_imagej_compatible

from stardist import random_label_cmap, _draw_polygons, export_imagej_rois
from stardist.models import StarDist2D

np.random.seed(6)
lbl_cmap = random_label_cmap()
from ipywidgets import interactive, interact
import os
import pandas as pd
from skimage.measure import regionprops_table
import seaborn as sns
from tqdm.notebook import tqdm
from skimage.transform import resize
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
from joblib import delayed, Parallel
from scipy.signal import find_peaks
from scipy.ndimage import gaussian_filter
from natsort import natsorted
from skimage.segmentation import clear_border, find_boundaries
from skimage.morphology import opening, erosion
from scipy.signal import find_peaks

In [ ]:
model_dir = "/home/georgeos/Storage/Dropbox (Cambridge University)/Phase/models/bent_noisematched_allsize_empty_und6_unarelu_unlasigmoid_epc2000_exp/"
model = StarDist2D(None, name='stardist_no_shape_completion', basedir=model_dir)

In [ ]:
image_dirs = glob("/home/georgeos/Storage/Dropbox (Cambridge University)/Phase/real_data/*.tif")

In [ ]:

cell_data = pd.DataFrame()
properties = ["area","centroid","major_axis_length","minor_axis_length","label","orientation"]
data = imread(image_dirs[3])[:500,:256,5:37]#[:,:256,5:37]
X = []
for x in range(len(data)):
    X.append(data[x])

requires_resize = True
resize_shape = (256,64) # (320,64) (512,128) (256,64)
if requires_resize:
    #X = [resize(x,(80,16), order=1, mode='reflect', cval=0, clip=True, preserve_range=True, anti_aliasing=None, anti_aliasing_sigma=None) for x in tqdm(X)]
    X = [resize(x,resize_shape, order=1, mode='reflect', cval=0, clip=True, preserve_range=True, anti_aliasing=None, anti_aliasing_sigma=None) for x in tqdm(X)]

n_channel = 1 if X[0].ndim == 2 else X[0].shape[-1]
axis_norm = (0,1)   # normalize channels independently
# axis_norm = (0,1,2) # normalize channels jointly
if n_channel > 1:
    print("Normalizing image channels %s." % ('jointly' if axis_norm is None or 2 in axis_norm else 'independently'))
cell_data_temp = pd.DataFrame()
for x in tqdm(range(len(X))):
    img = normalize(X[x], 1,99.8, axis=axis_norm)
    labels, details = model.predict_instances(img, prob_thresh=0.5,nms_thresh=0.1)
    #labels = resize(labels,(256,32), order=0, mode='reflect', cval=0, clip=True, preserve_range=True, anti_aliasing=None, anti_aliasing_sigma=None).astype(int)
    #img = resize(img,(256,32), order=1, mode='reflect', cval=0, clip=True, preserve_range=True, anti_aliasing=None, anti_aliasing_sigma=None)
    #nms_labels, details = model.predict_instances(img, prob_thresh=0.3,nms_thresh=0.3)
    border_cleared = clear_border(labels,buffer_size=4).astype(bool)*labels
    #border_cleared_nms = clear_border(nms_labels,buffer_size=5).astype(bool)*nms_labels
    all_labels.append(border_cleared)
    #all_labels_nms.append(border_cleared_nms)
    all_imgs.append(img)
    current_cell_data  = pd.DataFrame(regionprops_table(border_cleared,properties = properties)).sort_values(by = "centroid-0").iloc[0]
    current_cell_data["time"] = int(x)
    cell_data_temp = cell_data_temp.append(current_cell_data)
cell_data_temp["trench"] = t
cell_data = cell_data.append(cell_data_temp)


In [ ]:
plt.figure(figsize=(35,3))
plt.plot(cell_data.reset_index()["major_axis_length"][:600]*0.065*(42/64))
plt.ylabel("Length (micron)")
plt.xlabel("Frame")
##5% val

In [ ]:
@interact(x=(0,499))
def f(x):
    img = normalize(X[x], 1,99.8, axis=axis_norm)
    labels, details = model.predict_instances(img, prob_thresh=0.2,nms_thresh=0.01)
    #labels = resize(labels,(256,32), order=0, mode='reflect', cval=0, clip=True, preserve_range=True, anti_aliasing=None, anti_aliasing_sigma=None).astype(int)
    #img = resize(img,(256,32), order=1, mode='reflect', cval=0, clip=True, preserve_range=True, anti_aliasing=None, anti_aliasing_sigma=None)
    plt.figure(figsize=(8,8))
    plt.imshow(img if img.ndim==2 else img[...,0], clim=(0,1), cmap='gray')
    plt.imshow(labels, cmap=lbl_cmap, alpha=0.3)
    plt.axis('off');
    plt.show()
    plt.close()

In [ ]:

cell_data = pd.DataFrame()
properties = ["area","centroid","major_axis_length","minor_axis_length","label","orientation"]
for t in range(len(image_dirs)):
    data = imread(image_dirs[t])[:500,:256]#[:,:256,5:37]
    X = []
    for x in range(len(data)):
        X.append(data[x])

    requires_resize = True
    resize_shape = (320,64) # (320,64) (512,128)
    if requires_resize:
        #X = [resize(x,(80,16), order=1, mode='reflect', cval=0, clip=True, preserve_range=True, anti_aliasing=None, anti_aliasing_sigma=None) for x in tqdm(X)]
        X = [resize(x,resize_shape, order=1, mode='reflect', cval=0, clip=True, preserve_range=True, anti_aliasing=None, anti_aliasing_sigma=None) for x in tqdm(X)]

    n_channel = 1 if X[0].ndim == 2 else X[0].shape[-1]
    axis_norm = (0,1)   # normalize channels independently
    # axis_norm = (0,1,2) # normalize channels jointly
    if n_channel > 1:
        print("Normalizing image channels %s." % ('jointly' if axis_norm is None or 2 in axis_norm else 'independently'))
    try:
        cell_data_temp = pd.DataFrame()
        for x in tqdm(range(len(X))):
            img = normalize(X[x], 1,99.8, axis=axis_norm)
            labels, details = model.predict_instances(img, prob_thresh=0.5,nms_thresh=0.1)
            #labels = resize(labels,(256,32), order=0, mode='reflect', cval=0, clip=True, preserve_range=True, anti_aliasing=None, anti_aliasing_sigma=None).astype(int)
            #img = resize(img,(256,32), order=1, mode='reflect', cval=0, clip=True, preserve_range=True, anti_aliasing=None, anti_aliasing_sigma=None)
            #nms_labels, details = model.predict_instances(img, prob_thresh=0.3,nms_thresh=0.3)
            border_cleared = clear_border(labels,buffer_size=4).astype(bool)*labels
            #border_cleared_nms = clear_border(nms_labels,buffer_size=5).astype(bool)*nms_labels
            all_labels.append(border_cleared)
            #all_labels_nms.append(border_cleared_nms)
            all_imgs.append(img)
            current_cell_data  = pd.DataFrame(regionprops_table(border_cleared,properties = properties)).sort_values(by = "centroid-0").iloc[0]
            current_cell_data["time"] = int(x)
            cell_data_temp = cell_data_temp.append(current_cell_data)
        cell_data_temp["trench"] = t
        cell_data = cell_data.append(cell_data_temp)
    except:
        pass

In [ ]:
trenches = cell_data["trench"].unique()

In [ ]:
deltas = []
birth_lengths = []
division_lengths = []
for trench in trenches:
    sawtooth = np.array(cell_data[cell_data["trench"] == trench]["major_axis_length"][::3].reset_index(drop=True))
    top_peaks = find_peaks(sawtooth, prominence=20,distance=3)[0]
    bot_peaks = find_peaks(-sawtooth, prominence=20,distance=3)[0]
    #plt.figure(figsize=(24,4))
    #plt.plot(sawtooth)
    #plt.scatter(top_peaks, sawtooth[top_peaks],c="g")
    #plt.scatter(bot_peaks, sawtooth[bot_peaks],c="r")
    #plt.show()
    peak_pairs = []
    if top_peaks[0] - bot_peaks[0] > 0:
        for x in range(len(top_peaks)-1):
            peak_pairs.append([top_peaks[x+1],bot_peaks[x]])
    elif top_peaks[0] - bot_peaks[0] < 0:
        for x in range(len(bot_peaks)-1):
            peak_pairs.append([top_peaks[x],bot_peaks[x]])
    for pair in peak_pairs:
        delta = sawtooth[pair[0]] - sawtooth[pair[1]]
        deltas.append(delta)
    for peak in top_peaks:
        division_lengths.append(sawtooth[peak])
    for peak in bot_peaks:
        birth_lengths.append(sawtooth[peak])

In [ ]:
plt.hist(np.array(deltas)*0.065,bins=30)
plt.title("$\Delta$, $\mu$m")
plt.show()

In [ ]:
plt.hist(cell_data["major_axis_length"]*0.065,bins=30)
plt.show()

In [ ]:
plt.hist(np.array(birth_lengths)*0.065,bins=30)
plt.title("Birth length, $\mu$m")
plt.show()

In [ ]:
plt.hist(np.array(division_lengths)*0.065,bins=30)
plt.title("Birth length, $\mu$m")
plt.show()

In [ ]:
len(birth_lengths)

In [ ]:
len(division_lengths)

In [ ]:
plt.scatter(birth_lengths,division_lengths)

In [ ]:
all_labels = []
all_labels_nms = []
all_imgs = []
cell_data = pd.DataFrame()
properties = ["area","centroid","major_axis_length","minor_axis_length","label","orientation"]
for x in tqdm(range(len(X))):
    img = normalize(X[x], 1,99.8, axis=axis_norm)
    labels, details = model.predict_instances(img, prob_thresh=0.5,nms_thresh=0.1)
    #labels = resize(labels,(256,32), order=0, mode='reflect', cval=0, clip=True, preserve_range=True, anti_aliasing=None, anti_aliasing_sigma=None).astype(int)
    #img = resize(img,(256,32), order=1, mode='reflect', cval=0, clip=True, preserve_range=True, anti_aliasing=None, anti_aliasing_sigma=None)
    #nms_labels, details = model.predict_instances(img, prob_thresh=0.3,nms_thresh=0.3)
    border_cleared = clear_border(labels,buffer_size=4).astype(bool)*labels
    #border_cleared_nms = clear_border(nms_labels,buffer_size=5).astype(bool)*nms_labels
    all_labels.append(border_cleared)
    #all_labels_nms.append(border_cleared_nms)
    all_imgs.append(img)
    
    cell_data = cell_data.append(pd.DataFrame(regionprops_table(border_cleared,properties = properties)).sort_values(by = "centroid-0").iloc[0])
cell_data.reset_index(inplace=True)


In [ ]:
def get_true_mother(x):
    all_labels_converted = (all_labels[x].astype(bool) * all_labels_nms[x].astype(bool))*all_labels_nms[x]
    for lab in np.unique(all_labels_converted):
        if np.sum(all_labels_converted == lab) < 15:
            all_labels_converted = np.where(all_labels_converted != lab, all_labels_converted, 0)
    mother_cell_data = pd.DataFrame(regionprops_table(all_labels_converted,properties = properties)).sort_values(by = "centroid-0").iloc[0]
    mother_cell_mask = all_labels_converted == mother_cell_data["label"]
    mother_cell_border = find_boundaries(mother_cell_mask,mode="outer")
    nms_mask_labels = np.unique(mother_cell_border*(all_labels_nms[x]))
    nms_mask_labels = np.setdiff1d(nms_mask_labels,np.unique(all_labels_converted))
    nms_mask_labels = nms_mask_labels[nms_mask_labels > 0]
    border_nms = (find_boundaries(mother_cell_mask,mode="outer")*all_labels_nms[x])
    entire_mother = np.zeros(mother_cell_mask.shape)
    entire_mother += mother_cell_mask
    for lab in nms_mask_labels:
        if np.sum(border_nms == lab) > 30: 
            entire_mother += (all_labels_nms[x] == lab)
    return entire_mother

In [ ]:
mothers = []
mother_data = pd.DataFrame()
for x in range(len(all_labels_nms)):
    true_mother = get_true_mother(x).astype(int)
    mothers.append(true_mother)
    mother_data = mother_data.append(pd.DataFrame(regionprops_table(true_mother,properties = properties)))

In [ ]:
mother_data.reset_index(inplace=True)

In [ ]:
plt.figure(figsize=(35,3))
plt.plot(mother_data["major_axis_length"][:600]*0.065)
plt.ylabel("Length (micron)")
plt.xlabel("Frame")
##5% val

In [ ]:
plt.figure(figsize=(35,3))
plt.plot(cell_data.reset_index()["minor_axis_length"][:600]*0.065*(42/64))
plt.ylabel("Length (micron)")
plt.xlabel("Frame")
##5% val

In [ ]:
plt.figure(figsize=(35,3))
plt.plot(cell_data.reset_index()["major_axis_length"][:600]*0.065*(42/64))
plt.ylabel("Length (micron)")
plt.xlabel("Frame")
##5% val

In [ ]:
plt.hist(cell_data.reset_index()["minor_axis_length"][:600]*0.065*(42/64),bins = 20)
plt.show()

In [ ]:
Y = sorted(glob("/home/georgeos/Storage/Dropbox (Cambridge University)/Phase/cropped_histmatched/masks/*.tif"))
Y = list(map(imread,Y))
n_channel = 1 if X[0].ndim == 2 else X[0].shape[-1]
requires_resize = False
resize_shape = (256,32)
if requires_resize:
    Y = [resize(y,resize_shape, order=0, mode='reflect', cval=0, clip=True, preserve_range=True, anti_aliasing=None, anti_aliasing_sigma=None).astype(int) for y in tqdm(Y)]

In [ ]:
syn_mothers = pd.DataFrame()
for y in range(len(cell_data)):
    syn_mothers = syn_mothers.append(pd.DataFrame(regionprops_table(Y[y],properties = properties)).sort_values(by = "centroid-0").iloc[0])

In [ ]:
syn_mothers = syn_mothers.reset_index()
syn_mothers = syn_mothers[syn_mothers["orientation"] > -0.2]

In [ ]:
syn_mothers = syn_mothers[syn_mothers["orientation"] < 0.2]

In [ ]:
plt.hist(cell_data["orientation"]/(cell_data["major_axis_length"]),bins = 20,alpha=0.4)
plt.hist(syn_mothers["orientation"]/syn_mothers["major_axis_length"],bins = 28,alpha=0.4)
plt.title("Orientation/major_length")
plt.legend(["Real","Synthetic"])
plt.show()
plt.hist(cell_data["orientation"]/(cell_data["minor_axis_length"]),bins = 20,alpha=0.4)
plt.hist(syn_mothers["orientation"]/syn_mothers["minor_axis_length"],bins = 28,alpha=0.4)
plt.title("Orientation/minor_length")
plt.legend(["Real","Synthetic"])
plt.show()


In [ ]:
plt.hist(syn_mothers["orientation"]/syn_mothers["major_axis_length"],bins = 20)
plt.show()

In [ ]:
plt.figure(figsize=(35,3))
plt.plot(cell_data.reset_index()["major_axis_length"][::2][:600]*0.065)
plt.ylabel("Length (micron)")
plt.xlabel("Frame")
##5% val

In [ ]:
from scipy.signal import find_peaks

In [ ]:
plt.figure(figsize=(35,3))
plt.plot(cell_data.reset_index()["major_axis_length"][:600]*0.065*(256/128))
plt.ylabel("Length (micron)")
plt.xlabel("Frame")
##5% val

In [ ]:
@interact(x=(0,len(mothers)-1))
def f(x):
    plt.imshow(mothers[x])
    plt.show()
    plt.close()

In [ ]:
for x in tqdm(range(len(all_labels))):
    f, ax = plt.subplots(1,2,figsize=(3,7))
    ax[0].imshow(all_imgs[x],cmap="Greys_r")
    ax[1].imshow(all_labels[x],cmap=lbl_cmap)
    ax[1].axis("off")
    ax[0].axis("off")
    plt.savefig(model_dir+"/diagnostics/{}.jpeg".format(str(x).zfill(4)))
    plt.tight_layout()
    plt.close()

In [ ]:
growth_data = np.array(cell_data.reset_index()["major_axis_length"])
fig,ax = plt.subplots(figsize=(35,3))
ax.plot(np.diff(np.log2(growth_data)))

In [ ]:
growth_rates = np.diff(np.log2(growth_data))

In [ ]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

In [ ]:
fig,ax = plt.subplots(figsize=(35,3))
plt.plot(moving_average(np.where((growth_rates > 0) & (growth_rates < 0.1), growth_rates, 0),5))

In [ ]:
fig,ax = plt.subplots(figsize=(35,3))
ax.plot(cell_data.reset_index()["minor_axis_length"][:]*0.065*(42/64),alpha=0.6)
ax2=ax.twinx()
ax2.plot(cell_data.reset_index()["major_axis_length"][::2]*0.065,c="r",alpha=0.6)
ax.set_ylabel("Width (micron)")
ax2.set_ylabel("Length (micron)")
#plt.savefig("/home/georgeos/testtwin3.pdf")

In [ ]:
plt.scatter(cell_data.reset_index()["minor_axis_length"][:]*0.065*(42/64),cell_data.reset_index()["major_axis_length"][:]*0.065)
plt.ylabel("Length")

In [ ]:
import seaborn as sns
sns.set_theme(style="ticks")

rs = np.random.RandomState(11)
x = cell_data.reset_index()["minor_axis_length"][:]*0.065*(42/64)
y = cell_data.reset_index()["major_axis_length"][:]*0.065

sns.jointplot(x=x, y=y, color="#4CB391")

In [ ]:
plt.figure(figsize=(20,4))
plt.plot(np.convolve(x, np.ones(15)/15, mode='valid')-x[7:-7])


In [ ]:
np.std(np.convolve(x, np.ones(15)/15, mode='valid')-x[7:-7])

In [ ]:
plt.figure(figsize=(20,1))
plt.plot(cell_data.reset_index()["minor_axis_length"])

In [ ]:
output_dir = "/home/georgeos/Storage/Dropbox (Cambridge University)/PhD_Georgeos_Hardo/ML_based_segmentation_results/40x_Ph2_test_1.5/predictions_0.2"
try:
    os.mkdir(output_dir)
except:
    pass
try:
    os.mkdir(output_dir+"/masks_resized/")
    os.mkdir(output_dir+"/images_resized/")
except:
    pass
try:
    os.mkdir(output_dir+"/diagnostics/")
except:
    pass
try:
    os.mkdir(output_dir+"/postprocessed/")
except:
    pass

In [ ]:
all_labels = []
all_details = []
for z in tqdm(range(len(X))):
    img = normalize(X[z], 1,100, axis=axis_norm)
    labels, details = model.predict_instances(img, prob_thresh=0.2, nms_thresh=0.01)
    imsave(
        output_dir+"/masks_resized/{}".format(names[z]),labels)
    imsave(
        output_dir+"/images_resized/{}".format(names[z]),img)
    all_labels.append(labels)
    all_details.append(details)

In [ ]:
a = 0
for trench in tqdm(trench_names):
    idxs = trenches_idx[trench]
    for z in (idxs):
        cleared_mask = clear_side_masks(all_labels[z],a)
        imsave(output_dir+"/postprocessed/{}".format(names[z]),cleared_mask)
    a+=1

In [ ]:
pp = sorted(glob("/home/georgeos/Storage/Dropbox (Cambridge University)/PhD_Georgeos_Hardo/ML_based_segmentation_results/40x_Ph2_test_1.5/predictions_0.2/postprocessed/*"))
lbls = sorted(glob("/home/georgeos/Storage/Dropbox (Cambridge University)/PhD_Georgeos_Hardo/ML_based_segmentation_results/40x_Ph2_test_1.5/predictions_0.2/postprocessed/*"))


def plotter(z): 
    labels = all_labels[z]
    details = all_details[z]
    img = X[z]
    probmap = np.zeros(labels.shape)
    for x in range(len(details["points"])):
        probmap[np.where(labels == labels[details["points"][x][0],details["points"][x][1]])] = details["prob"][x]
    f, ax = plt.subplots(1,5,figsize=(7.9,4))
    ax[0].imshow(img,cmap="Greys_r")
    ax[1].imshow(labels,cmap=lbl_cmap)
    prob = ax[3].imshow(probmap,cmap="afmhot",vmin=0,vmax=1)
    ax[2].imshow(img,cmap="Greys_r")
    ax[2].imshow(labels,cmap=lbl_cmap,alpha=0.45)
    ax[2].set_title("PC+Masks")
    f.colorbar(prob,ax=ax[3])
    ax[3].set_title("P(Mask)")
    ax[1].set_title("Masks")
    ax[0].set_title("PC_image")
    ax[4].imshow(imread(pp[z]),cmap=lbl_cmap)
    ax[4].set_title("Post-proc")
    for axs in ax:
        axs.axis("off")
    plt.tight_layout()
    plt.savefig(output_dir+"/diagnostics/{}".format(names[z][:-3]+"jpeg"),dpi=100)
    plt.close()

In [ ]:
Parallel(n_jobs=1)(delayed(plotter)(i) for i in tqdm(range(len(all_labels)))) #plotter is leaking memory, use n_jobs = 1 for now

In [ ]:
output_dir = "/home/georgeos/Storage/Dropbox (Cambridge University)/PhD_Georgeos_Hardo/ML_based_segmentation_results/40x_Ph2_test_1.5/predictions_0.1"
try:
    os.mkdir(output_dir)
except:
    pass
try:
    os.mkdir(output_dir+"/masks_resized/")
    os.mkdir(output_dir+"/images_resized/")
except:
    pass
try:
    os.mkdir(output_dir+"/diagnostics/")
except:
    pass
try:
    os.mkdir(output_dir+"/postprocessed/")
except:
    pass

In [ ]:
all_labels = []
all_details = []
for z in tqdm(range(len(X))):
    img = normalize(X[z], 1,100, axis=axis_norm)
    labels, details = model.predict_instances(img, prob_thresh=0.1, nms_thresh=0.01)
    imsave(
        output_dir+"/masks_resized/{}".format(names[z]),labels)
    imsave(
        output_dir+"/images_resized/{}".format(names[z]),img)
    all_labels.append(labels)
    all_details.append(details)

In [ ]:
a = 0
for trench in tqdm(trench_names):
    idxs = trenches_idx[trench]
    for z in (idxs):
        cleared_mask = clear_side_masks(all_labels[z],a)
        imsave(output_dir+"/postprocessed/{}".format(names[z]),cleared_mask)
    a+=1

In [ ]:
pp = sorted(glob("/home/georgeos/Storage/Dropbox (Cambridge University)/PhD_Georgeos_Hardo/ML_based_segmentation_results/40x_Ph2_test_1.5/predictions_0.1/postprocessed/*"))

def plotter(z): 
    labels = all_labels[z]
    details = all_details[z]
    img = X[z]
    probmap = np.zeros(labels.shape)
    for x in range(len(details["points"])):
        probmap[np.where(labels == labels[details["points"][x][0],details["points"][x][1]])] = details["prob"][x]
    f, ax = plt.subplots(1,5,figsize=(7.9,4))
    ax[0].imshow(img,cmap="Greys_r")
    ax[1].imshow(labels,cmap=lbl_cmap)
    prob = ax[3].imshow(probmap,cmap="afmhot",vmin=0,vmax=1)
    ax[2].imshow(img,cmap="Greys_r")
    ax[2].imshow(labels,cmap=lbl_cmap,alpha=0.45)
    ax[2].set_title("PC+Masks")
    f.colorbar(prob,ax=ax[3])
    ax[3].set_title("P(Mask)")
    ax[1].set_title("Masks")
    ax[0].set_title("PC_image")
    ax[4].imshow(imread(pp[z]),cmap=lbl_cmap)
    ax[4].set_title("Post-proc")
    for axs in ax:
        axs.axis("off")
    plt.tight_layout()
    plt.savefig(output_dir+"/diagnostics/{}".format(names[z][:-3]+"jpeg"),dpi=100)
    plt.close()

In [ ]:
Parallel(n_jobs=1)(delayed(plotter)(i) for i in tqdm(range(len(all_labels)))) #plotter is leaking memory, use n_jobs = 1 for now

## playing around

In [ ]:
all_labels_1 = []
all_details_1 = []
all_imgs_1 = []
for z in range(len(X)):
    img = normalize(X[z], 1,100, axis=axis_norm)
    labels, details = model.predict_instances(img, prob_thresh=0.7, nms_thresh=0.1)
    all_labels_1.append(labels)
    all_details_1.append(details)
    all_imgs_1.append(img)

@interact
def plotter_1(x=(0,len(X)-1)):
    plt.figure(figsize=(8,8))
    plt.imshow(all_imgs_1[x] if all_imgs_1[x].ndim==2 else all_imgs_1[x][...,0], clim=(0,1), cmap='gray')
    plt.imshow(all_labels_1[x], cmap=lbl_cmap, alpha=0.35)
    plt.axis('off');
    plt.show()
    plt.close()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(resize(all_imgs_1[0],(211,40), order=0, mode='reflect', cval=0, clip=True, preserve_range=True, anti_aliasing=None, anti_aliasing_sigma=None),cmap="Greys_r")

In [ ]:
from scipy.signal import find_peaks
from skimage.segmentation import clear_border

In [ ]:
properties = ["area","centroid","major_axis_length","minor_axis_length"]

trenches = os.listdir("/home/georgeos/Storage/Dropbox (Cambridge University)/PhD_Georgeos_Hardo/ML_based_segmentation_results/40x_Ph2_test_1.5/top_trenches_PC/")
trenches.sort()
model = StarDist2D(None, name='stardist_no_shape_completion', basedir='models_mixed_size_1600_epoch_good_50')
all_cell_data = pd.DataFrame()
master_label = []
master_imgs = []
exception_trenches = ["trench_52", "trench_54"]
for trench in tqdm(trenches):
    cell_data = pd.DataFrame()

    X = sorted(glob("/home/georgeos/Storage/Dropbox (Cambridge University)/PhD_Georgeos_Hardo/ML_based_segmentation_results/40x_Ph2_test_1.5/top_trenches_PC/{}/*.tif".format(trench)))
    X = list(map(imread,X))

    n_channel = 1 if X[0].ndim == 2 else X[0].shape[-1]
    axis_norm = (0,1)   # normalize channels independently
    # axis_norm = (0,1,2) # normalize channels jointly
    if n_channel > 1:
        print("Normalizing image channels %s." % ('jointly' if axis_norm is None or 2 in axis_norm else 'independently'))


    all_labels = []
    all_details = []
    all_imgs = []
    for z in range(3,25):
        img = normalize(X[z], 1,100, axis=axis_norm)
        if z == 3:
            if trench in exception_trenches:
                midpoint = int(img.shape[1]/2)
            else:
                image_profile = img.mean(axis=0)
                peaks = find_peaks(image_profile, height=np.mean(image_profile)*0.9)
                if len(peaks[0]) > 1:
                    midpoint = int(np.sum(peaks[0])/2)
                elif len(peaks[0]) == 1:
                    midpoint = peaks[0][0]
                trench_half_width = 15
        
        labels, details = model.predict_instances(img, prob_thresh=0.7, nms_thresh=0.1)
        

        labels = clear_border(labels[:,midpoint-trench_half_width:midpoint+trench_half_width])
        
        
        all_labels.append(labels)
        all_details.append(details)
        all_imgs.append(img[:,midpoint-trench_half_width:midpoint+trench_half_width])
        
    for label_image in all_labels:
        if len(np.unique(label_image)) > 1:
            cell_data = cell_data.append(pd.DataFrame(regionprops_table(label_image=label_image, properties=properties)).sort_values(by = "centroid-0").iloc[0])
        else:
            cell_data = cell_data.append(pd.Series([np.nan]*len(properties)),ignore_index=True)
    cell_data.reset_index(inplace=True,drop=True)
    cell_data["cell"] = trench
    cell_data["time"] = range(3,25)
    all_cell_data = all_cell_data.append(cell_data)
    all_cell_data.reset_index(inplace=True,drop=True)
    master_label.append(all_labels)
    master_imgs.append(all_imgs)

In [ ]:
g = sns.FacetGrid(all_cell_data, col='cell', col_wrap=7)
g = g.map(sns.lineplot, 'time', 'major_axis_length', ci=None).add_legend()
plt.tight_layout()
plt.savefig("test_1600_epoch_50perc.pdf")

In [ ]:
for x in range(len(master_imgs[0])):
    plt.imshow(master_imgs[54][x],cmap="Greys_r")
    plt.imshow(master_label[54][x],alpha=0.4,cmap="jet")
    plt.show()
    plt.close()

In [ ]:
if False:
    fig, ax = plt.subplots(7,8, figsize=(16,16))
    for i,(a,x) in enumerate(zip(ax.flat, X)):
        a.imshow(x if x.ndim==2 else x[...,0], cmap='gray')
        a.set_title(i)
    [a.axis('off') for a in ax.flat]
    plt.tight_layout()
None;

In [ ]:
all_labels = []
all_details = []
all_imgs = []
for z in range(25):
    img = normalize(X[z], 1,100, axis=axis_norm)
    labels, details = model.predict_instances(img, prob_thresh=0.6, nms_thresh=0.1)
    all_labels.append(labels)
    all_details.append(details)
    all_imgs.append(img)
    
@interact
def plotter(x=(0,len(X)-1)):
    plt.figure(figsize=(8,8))
    plt.imshow(all_imgs[x] if all_imgs[x].ndim==2 else all_imgs[x][...,0], clim=(0,1), cmap='gray')
    plt.imshow(all_labels[x], cmap=lbl_cmap, alpha=0.35)
    plt.axis('off');
    plt.show()
    plt.close()

In [ ]:
from skimage.measure import regionprops, regionprops_table
import pandas as pd

In [ ]:
trenches = os.listdir("/home/georgeos/Storage/Dropbox (Cambridge University)/PhD_Georgeos_Hardo/ML_based_segmentation_results/40x_Ph2_test_1.5/top_trenches_PC/")

In [ ]:
a = 0
for t in range(len(trench))
cell_data = pd.DataFrame()
for label_image in all_labels:
    cell_data = cell_data.append(pd.DataFrame(regionprops_table(label_image=label_image, properties=properties)).sort_values(by = "centroid-0").iloc[0])
cell_data.reset_index(inplace=True,drop=True)
cell_data["cell"] = a

In [ ]:
cell_data

In [ ]:
model = StarDist2D(None, name='stardist_no_shape_completion', basedir='models_nohist')

trenches = os.listdir("/home/georgeos/Storage/Dropbox (Cambridge University)/PhD_Georgeos_Hardo/ML_based_segmentation_results/40x_Ph2_test_1.5/top_trenches_PC/")
all_cell_data = pd.DataFrame()
for trench in trenches:
    cell_data = pd.DataFrame()

    X = sorted(glob("/home/georgeos/Storage/Dropbox (Cambridge University)/PhD_Georgeos_Hardo/ML_based_segmentation_results/40x_Ph2_test_1.5/top_trenches_PC/{}/*.tif".format(trench)))
    X = list(map(imread,X))

    n_channel = 1 if X[0].ndim == 2 else X[0].shape[-1]
    axis_norm = (0,1)   # normalize channels independently
    # axis_norm = (0,1,2) # normalize channels jointly
    if n_channel > 1:
        print("Normalizing image channels %s." % ('jointly' if axis_norm is None or 2 in axis_norm else 'independently'))


    all_labels = []
    all_details = []
    all_imgs = []
    for z in range(0,25):
        img = normalize(X[z], 1,100, axis=axis_norm)
        labels, details = model.predict_instances(img, prob_thresh=0.7, nms_thresh=0.1)
        all_labels.append(labels)
        all_details.append(details)
        all_imgs.append(img)
        
    for label_image in all_labels:
        cell_data = cell_data.append(pd.DataFrame(regionprops_table(label_image=label_image, properties=properties)).sort_values(by = "centroid-0").iloc[0])
    cell_data.reset_index(inplace=True,drop=True)
    cell_data["cell"] = trench
    cell_data["time"] = range(0,25)
    all_cell_data = all_cell_data.append(cell_data)
    all_cell_data.reset_index(inplace=True,drop=True)
    print(trench)


In [ ]:
all_cell_data["minor_axis_length"] = (all_cell_data["minor_axis_length"])*2.5

In [ ]:
all_cell_data_temp = pd.melt(all_cell_data,value_vars=["major_axis_length","minor_axis_length"],id_vars =["centroid-0","centroid-1","cell","time"])

In [ ]:
g = sns.FacetGrid(all_cell_data_temp, col='cell', hue="variable",col_wrap=7)
g = g.map(sns.lineplot, 'time', "value", ci=None).add_legend()
plt.tight_layout()
plt.savefig("test_nohist.pdf",dpi=200)

In [ ]:
model = StarDist2D(None, name='stardist_no_shape_completion', basedir='models')

trenches = os.listdir("/home/georgeos/Storage/Dropbox (Cambridge University)/PhD_Georgeos_Hardo/ML_based_segmentation_results/40x_Ph2_test_1.5/top_trenches_PC/")
all_cell_data = pd.DataFrame()
for trench in trenches:
    cell_data = pd.DataFrame()

    X = sorted(glob("/home/georgeos/Storage/Dropbox (Cambridge University)/PhD_Georgeos_Hardo/ML_based_segmentation_results/40x_Ph2_test_1.5/top_trenches_PC/{}/*.tif".format(trench)))
    X = list(map(imread,X))

    n_channel = 1 if X[0].ndim == 2 else X[0].shape[-1]
    axis_norm = (0,1)   # normalize channels independently
    # axis_norm = (0,1,2) # normalize channels jointly
    if n_channel > 1:
        print("Normalizing image channels %s." % ('jointly' if axis_norm is None or 2 in axis_norm else 'independently'))


    all_labels = []
    all_details = []
    all_imgs = []
    for z in range(0,25):
        img = normalize(X[z], 1,100, axis=axis_norm)
        labels, details = model.predict_instances(img, prob_thresh=0.65, nms_thresh=0.1)
        all_labels.append(labels)
        all_details.append(details)
        all_imgs.append(img)
        
    for label_image in all_labels:
        cell_data = cell_data.append(pd.DataFrame(regionprops_table(label_image=label_image, properties=properties)).sort_values(by = "centroid-0").iloc[0])
    cell_data.reset_index(inplace=True,drop=True)
    cell_data["cell"] = trench
    cell_data["time"] = range(0,25)
    all_cell_data = all_cell_data.append(cell_data)
    all_cell_data.reset_index(inplace=True,drop=True)
    print(trench)
g = sns.FacetGrid(all_cell_data, col='cell', col_wrap=7)
g = g.map(sns.lineplot, 'time', 'major_axis_length', ci=None).add_legend()
plt.tight_layout()
plt.savefig("test_mixed_cells.png",dpi=200)

In [ ]:
all_labels_1 = []
all_details_1 = []
all_imgs_1 = []
for z in range(25):
    img = normalize(X[z], 1,100, axis=axis_norm)
    labels, details = model.predict_instances(img, prob_thresh=0.6, nms_thresh=0.1)
    all_labels_1.append(labels)
    all_details_1.append(details)
    all_imgs_1.append(img)
    
@interact
def plotter_1(x=(0,len(X)-1)):
    plt.figure(figsize=(8,8))
    plt.imshow(all_imgs_1[x] if all_imgs[x].ndim==2 else all_imgs_1[x][...,0], clim=(0,1), cmap='gray')
    plt.imshow(all_labels_1[x], cmap=lbl_cmap, alpha=0.35)
    plt.axis('off');
    plt.show()
    plt.close()